## Introduction

In this notebook, we will build a simple detection algorithm based on what we had already, and a useful new function called *findContours*. To be more precise, with binarization and morphological operations, we are now able to extract grayscale images with regions of interest (foreground) highlighted in white; while other regions (background) are black. Now we want to actually localize these objects of interest in the image.

We will experiment this algorithm on the light bulb detection problem we worked on a while ago. To start with, we need to reuse code from that part.

In [ ]:
import cv2
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
def my_imshow(img):
    if len(img.shape) == 3: # RGB image
        temp_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(temp_img)
    else: # black and white image
        plt.imshow(img, cmap = 'gray')

In [ ]:
img = cv2.imread('./imgs/bulbs_in_room.jpg')
my_imshow(img)

Remember we also built a binarization algorithm for that. We are going to need that as well.

In [ ]:
hard_cutoff = 252
# 1. Convert it to grayscale
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 2. Apply hard cut off
img_binarized = (img_gray > hard_cutoff)
# 3. Scale for display purpose
img_binarized = (img_binarized * 255).astype(np.uint8)
# 4. Visually inspect our result
plt.imshow(img_binarized, cmap = 'gray')

### How cv2.findContours works

I don't want to get into the dirty details of how exactly findContours was constructed, but the big idea here is that findContours attempts to capture geometric objects in the image. Let's see it in action!

In [ ]:
contours, hierarchy = cv2.findContours(img_binarized, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
drawn_img = cv2.drawContours(cv2.cvtColor(img_binarized, cv2.COLOR_GRAY2BGR), contours, -1, (0, 255, 0), 3)
my_imshow(drawn_img)

In [ ]:
# Visualize it on original image
drawn_img = cv2.drawContours(img, contours, -1, (0, 255, 0), 3)
my_imshow(drawn_img)

... which is of pretty good quality! Recall that our task was to identify functioning light bulbs. The binarization threshold we found, combined with findContours on binarized image, does this job pretty well!

However, do note that, if you look close enough on the results visualized by the drawContours function above, you will notice that there are a few little green dots that do not belong to functioning light bulbs. This is not uncommon when we design CV algorithm in this approach. In fact, we can try,

In [ ]:
print("Number of functioning light bulbs: {}".format(len(contours)))

which makes absolutely nonsense!

Observe that, the real functioning light bulbs are usually emitting lights such that it occupies a large region in the binarized image, while the noises occupy little to no area. Recall the morphological operations from last notebook. We can apply those operations to get rid of the noises!

### Use 'Opening' followed by 'Closing' to get rid of noises

In [ ]:
kernel = np.ones((5, 5), np.uint8) # kernel size needs to be carefully chosen!
processed_img = cv2.morphologyEx(img_binarized, cv2.MORPH_OPEN, kernel)
processed_img = cv2.morphologyEx(processed_img, cv2.MORPH_CLOSE, kernel)
my_imshow(processed_img)

In [ ]:
# Try cv2.findContours again and visualize the result
contours, hierarchy = cv2.findContours(processed_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
drawn_img = cv2.drawContours(cv2.cvtColor(img_binarized, cv2.COLOR_GRAY2BGR), contours, -1, (0, 255, 0), 3)
my_imshow(drawn_img)

In [ ]:
# Visualize it on original image
drawn_img = cv2.drawContours(img, contours, -1, (0, 255, 0), 3)
print("Number of functioning light bulbs: {}".format(len(contours)))
my_imshow(drawn_img)

which capture the number of functioning light bulbs precisely. Well done!

### MP4.1

Take your code from MP3.1, in which you used binarization and morphological operations to isolate those light bars in the image. Now, use *cv2.findContours* and *cv2.drawContours* to detect these light bars.

In [ ]:
# Your code goes here

### MP4.2

Examine closer to elements in the *contours* variable. What does it mean? How are contours represented? After you understand how it works, design a function that outputs the center of these light bars.

Hint: compute averages of x/y coordinates.

In [ ]:
# Your code goes here